In [1]:
import collections
import numpy as np

In [2]:
from spn.algorithms import Inference
from spn.algorithms import Gradient

In [3]:
from spn.structure.Base import Leaf, bfs
from spn.structure.leaves.piecewise.PiecewiseLinear import PiecewiseLinear
from spn.structure.leaves.piecewise.Gradients import piecewise_gradient

In [4]:
piecewise_spn = (
    0.5 * PiecewiseLinear([0, 1, 2], [0, 1, 0], [], scope=[0])
    + 0.5 * PiecewiseLinear([-2, -1, 0], [0, 1, 0], [], scope=[0])
) * (
    0.5 * PiecewiseLinear([0, 1, 2], [0, 1, 0], [], scope=[1])
    + 0.5 * PiecewiseLinear([-1, 0, 1], [0, 1, 0], [], scope=[1])
)


In [5]:
lls_matrix = np.full((1,7),np.nan)
ll = Inference.log_likelihood(piecewise_spn, np.array([[0.5, 1.5]]), lls_matrix=lls_matrix)
print(lls_matrix)
print(np.exp(ll))

[[-2.77258872 -1.38629436 -1.38629436 -0.69314718        -inf -0.69314718
         -inf]]
[[0.0625]]


In [6]:
print(np.exp(lls_matrix))

[[0.0625 0.25   0.25   0.5    0.     0.5    0.    ]]


In [7]:
gradients = np.exp(Gradient.gradient_backward(piecewise_spn, lls_matrix))

In [8]:
def print_spn_nodes(spn):
    q = queue.Queue()
    def build_queue(node):
        if not isinstance(node, Leaf):
            q.put(node.children)
            for c in node.children:
                build_queue(c)
    q.put(spn)
    build_queue(spn)
    return q

In [9]:
delta = 0.0001
(Inference.likelihood(piecewise_spn, np.array([[0.5 + delta, 1.5]])) - Inference.likelihood(piecewise_spn, np.array([[0.5, 1.5]])))/delta
delta = 0.0001
(Inference.likelihood(piecewise_spn, np.array([[0.5, 1.5 + delta]])) - Inference.likelihood(piecewise_spn, np.array([[0.5, 1.5]])))/delta

array([[-0.125]])

In [10]:
def gradient_at_leaf(node, ll):
    return piecewise_gradient(node, ll)

In [11]:
x = np.array([[0.5, 1.5], [1.5, 0]])
#q = collections.deque()
#bfs(piecewise_spn, lambda x: q.append(x))
#res = np.full((gradients.shape[0], max(piecewise_spn.scope) + 1), 0.0)
#for n, g, ll in zip(q, gradients[0], np.exp(lls_matrix[0])):
#    if isinstance(n, Leaf):
#        print(n)
#        print(g * gradient_at_leaf(n, x))
#        res[:, n.scope] += g * gradient_at_leaf(n, x)
#res

In [12]:
from spn.algorithms.Gradient import feature_gradient

In [13]:
feature_gradient(piecewise_spn, x)

array([[ 0.125, -0.125],
       [-0.25 ,  0.125]])